In [1]:
### This script will plot some graphics showing the relative locations of the ZDR and KDP maxes in the storm.
import matplotlib.pyplot as plt
import pyart
import numpy as np
import numpy.ma as ma
from metpy.units import atleast_1d, check_units, concatenate, units
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from siphon.radarserver import RadarServer
rs = RadarServer('http://thredds-aws.unidata.ucar.edu/thredds/radarServer/nexrad/level2/S3/')
#rs = RadarServer('http://thredds.ucar.edu/thredds/radarServer/nexrad/level2/IDD/')
from datetime import datetime, timedelta
from siphon.cdmr import Dataset
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from metpy.units import atleast_1d, check_units, concatenate, units
from shapely.geometry import polygon as sp
import pyproj 
import shapely.ops as ops
from shapely.ops import transform
from shapely.geometry.polygon import Polygon
from functools import partial
from shapely import geometry
import netCDF4
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage import data, img_as_float
from pyproj import Geod
from metpy.calc import get_wind_dir


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



C:\Users\mwilson41\AppData\Local\Continuum\Anaconda3\envs\radar\lib\site-packages\urllib3\contrib\pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [9]:
storm_relative_dir = 76
srdir = 76
query = rs.query()
#Here, set the initial time of the archived radar loop you want.
dt = datetime(2017, 10, 8, 0, 0) # Our specified time
station = 'KBMX'
query.stations(station).time_range(dt, dt + timedelta(hours=3.5))
cat = rs.get_catalog(query)
cat.datasets
f = 27
n = 1
for item in sorted(cat.datasets.items()):
    # After looping over the list of sorted datasets, pull the actual Dataset object out
    # of our list of items and access over CDMRemote
    try:
        ds = item[1]
        radar1 = pyart.io.nexrad_cdm.read_nexrad_cdm(ds.access_urls['OPENDAP'])
        #Now let's calculate and plot specific differential phase.
        for i in range(radar1.nsweeps):
            print('in loop')
            print(radar1.nsweeps)
            radar = radar1.extract_sweeps([i])
            if ((np.mean(radar.elevation['data']) < .60) and (np.max(np.asarray(radar.fields['differential_reflectivity']['data'])) != np.min(np.asarray(radar.fields['differential_reflectivity']['data'])))):
                n = n+1
                print(np.mean(radar.elevation['data']))
                time_start = netCDF4.num2date(radar.time['data'][0], radar.time['units'])
                print(time_start)
                kdp_dict = pyart.retrieve.kdp_proc.kdp_maesaka(radar)
                print('its this line')
                radar.add_field('KDP', kdp_dict[0])
                print('heres the problem')
                # mask out last 10 gates of each ray, this removes the "ring" around the radar.
                radar.fields['differential_reflectivity']['data'][:, -10:] = np.ma.masked
                ref_ungridded = radar.fields['reflectivity']['data']
                refl_c = np.copy(ref_ungridded)
                ref_c = ma.masked_where(refl_c < 20., refl_c)
                #Get ungridded ZDR
                zdr_ungridded = radar.fields['differential_reflectivity']['data']
                zdrl_c = np.copy(zdr_ungridded)
                zdr_c = ma.masked_where(refl_c < 20, zdrl_c)

                ungrid_lons = radar.gate_longitude['data']
                ungrid_lats = radar.gate_latitude['data']
                # exclude masked gates from the gridding
                gatefilter = pyart.filters.GateFilter(radar)
                gatefilter.exclude_masked('differential_reflectivity')
                print('almost gridding')
                #Now let's grid the data
                grid = pyart.map.grid_from_radars(
                    (radar,), gatefilters=(gatefilter, ),
                    grid_shape=(1, 500, 500),
                    grid_limits=((200, 200), (-123000.0, 123000.0), (-123000.0, 123000.0)),
                    fields=['differential_reflectivity','reflectivity','KDP','cross_correlation_ratio'])
                #Get gate altitudes
                gate_altitude = radar.gate_altitude['data'][:]
                #Get the data from the grid
                ZDR = grid.fields['differential_reflectivity']['data'][0]
                REF = grid.fields['reflectivity']['data'][0]
                KDP = grid.fields['KDP']['data'][0]
                CC = grid.fields['cross_correlation_ratio']['data'][0]

                #Mask everything below 20dbz
                #import numpy.ma as ma
                ZDRmasked1 = ma.masked_where(REF < 20, ZDR)
                REFmasked = ma.masked_where(REF < 20, REF)
                KDPmasked = ma.masked_where(REF < 20, KDP)
                KDPmasked = ma.filled(KDPmasked, fill_value = -2)
                #Filter out spots not in forward flank using gradient direction
                print('made it to smoothing')
                smoothed_ref1 = ndi.gaussian_filter(REFmasked, sigma = 2, order = 0)
                REFgradient = np.asarray(np.gradient(smoothed_ref1))
                REFgradient[0,:,:] = ma.masked_where(REF < 20, REFgradient[0,:,:])
                REFgradient[1,:,:] = ma.masked_where(REF < 20, REFgradient[1,:,:])
                print('made it through gradient')
                grad_dir1 = get_wind_dir(REFgradient[1,:,:] * units('m/s'), REFgradient[0,:,:] * units('m/s'))
                grad_dir = ma.masked_where(REF < 20, grad_dir1)
                #srdir = storm_relative_dir
                grad_ffd = np.abs(np.arctan2(np.sin(grad_dir * units('degrees')-srdir * units('degrees')), np.cos(grad_dir * units('degrees')-srdir * units('degrees'))))
                grad_ffd = grad_ffd.to('degrees')
                print('got gradient')
                #Add a fill value for the ZDR mask so that contours will be closed
                ZDRmasked2 = ma.masked_where(grad_ffd > 120 * units('degrees'), ZDRmasked1)
                ZDRmasked = ma.masked_where(CC < .60, ZDRmasked2)
                ZDRmasked = ma.filled(ZDRmasked, fill_value = -2)

                rlons = grid.point_longitude['data']
                rlats = grid.point_latitude['data']
                rlons_2d = rlons[0,:,:]
                rlats_2d = rlats[0,:,:]
                cenlat = radar.latitude['data'][0]
                cenlon = radar.longitude['data'][0]
                #Let's set up the map projection!
                print('Set up our projection')
                crs = ccrs.LambertConformal(central_longitude=-100.0, central_latitude=45.0)

                # Set up our array of latitude and longitude values and transform to 
                # the desired projection.

                tlatlons = crs.transform_points(ccrs.LambertConformal(central_longitude=265, central_latitude=25, standard_parallels=(25.,25.)),rlons[0,:,:],rlats[0,:,:])
                tlons = tlatlons[:,:,0]
                tlats = tlatlons[:,:,1]

                # Limit the extent of the map area, must convert to proper coords.
                LL = (cenlon-1,cenlat-1,ccrs.PlateCarree())
                UR = (cenlon+1,cenlat+1,ccrs.PlateCarree())
                print(LL)

                # Get data to plot state and province boundaries
                states_provinces = cfeature.NaturalEarthFeature(
                        category='cultural',
                        name='admin_1_states_provinces_lakes',
                        scale='50m',
                        facecolor='none')
                fname = 'cb_2016_us_county_20m/cb_2016_us_county_20m.shp'
                fname2 = 'cb_2016_us_state_20m/cb_2016_us_state_20m.shp'
                counties = ShapelyFeature(Reader(fname).geometries(),ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black')
                states = ShapelyFeature(Reader(fname2).geometries(),ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black')
                fig=plt.figure(n,figsize=(30.,25.))
                ax = plt.subplot(111,projection=ccrs.PlateCarree())
                ax.coastlines('50m',edgecolor='black',linewidth=0.75)
                #ax.add_feature(states_provinces,edgecolor='black',linewidth=0.5)
                ax.add_feature(counties, edgecolor = 'black', linewidth = 0.5)
                ax.add_feature(states, edgecolor = 'black', linewidth = 1.5)
                ax.set_extent([LL[0],UR[0],LL[1],UR[1]])
                REFlevels = np.arange(20,73,2)
                print('plotting')
                #ax.contourf(rlons[0,:,:],rlats[0,:,:],REFmasked,REFlevels,cmap = plt.cm.gist_ncar)
                refp = ax.pcolormesh(ungrid_lons, ungrid_lats, ref_c, cmap=plt.cm.gist_ncar, vmin = 10, vmax = 73)
                #zdrp = ax.pcolormesh(ungrid_lons, ungrid_lats, zdr_c, cmap=plt.cm.nipy_spectral, vmin = -2, vmax = 6)
                #Plot local maxes in reflectivity
                # image_max is the dilation of im with a 20*20 structuring element
                # It is used within peak_local_max function
                smoothed_ref = ndi.gaussian_filter(REFmasked, sigma = 3, order = 0)
                #image_max = ndi.maximum_filter(smoothed_ref, size=15, mode='constant')

                # Comparison between image_max and im to find the coordinates of local maxima
                #coordinates = peak_local_max(smoothed_ref, min_distance=20)
                #ref_maxes = REFmasked[coordinates[:,0], coordinates[:,1]]
                #max_lons = rlons[0,coordinates[:,0], coordinates[:,1]]
                #max_lats = rlats[0,coordinates[:,0], coordinates[:,1]]
                #max_lons_c = max_lons[ref_maxes > 45]
                #max_lats_c = max_lats[ref_maxes > 45]
                
                REFlev = [40]
                REFlev1 = [45]
                refc = ax.contour(rlons[0,:,:],rlats[0,:,:],smoothed_ref,REFlev)

                ref_areas = []
                max_lons_c = []
                max_lats_c = []
                
                proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
                           pyproj.Proj(init='epsg:3857'))

                #Look for reflectivity centroids
                for col in refc.collections:
                    # Loop through all polygons that have the same intensity level
                    for contour_path in col.get_paths(): 
                        # Create the polygon for this intensity level
                        # The first polygon in the path is the main one, the following ones are "holes"
                        for ncp,cp in enumerate(contour_path.to_polygons()):
                            print(1)
                            cpa = np.asarray(cp[:])
                            x = cpa[:,0]
                            y = cpa[:,1]
                            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
                            if ncp == 0:
                                poly = new_shape
                            else:
                                # Remove the holes if there are any
                                poly = poly.difference(new_shape)

                        # do something with polygon
                        #print(poly.area) 
                        print(poly.centroid.x)
                        s_new = transform(proj, poly)
                        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
                        boundary = np.asarray(poly.boundary.xy)
                        polypath = Path(boundary.transpose())
                        coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
                        maskr = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                        meanr = np.mean(smoothed_ref[maskr])
                        print('past mask')
                        print(meanr)
                        if projected_area > 10 * units('km^2') and meanr > REFlev[0]:
                            print('found a storm')
                            #For big blobs with embedded supercells, find the embedded storm cores
                            if projected_area > 200 * units('km^2'):
                                print('found a big storm')
                                rlon_2 = rlons[0,:,:]
                                rlat_2 = rlats[0,:,:]
                                #smoothed_ref_m = ma.MaskedArray(smoothed_ref, mask=maskr)
                                smoothed_ref_m = ma.masked_where(maskr==False, smoothed_ref)
                                smoothed_ref_m = ma.filled(smoothed_ref_m, fill_value = -2)
                                rlon2m = ma.MaskedArray(rlon_2, mask=maskr)
                                rlat2m = ma.MaskedArray(rlat_2, mask=maskr)
                                refc1 = ax.contour(rlon2m,rlat2m,smoothed_ref_m,REFlev1, linewidths = 3, linestyle = '--')
                                #refc1 = ax.contour(rlon_2[maskr],rlat_2[maskr],smoothed_ref[maskr],REFlev1, colors = 'g', linewidths = 3)
                                print('plotted a big storm')
                                #Look for reflectivity centroids
                                for col1 in refc1.collections:
                                    # Loop through all polygons that have the same intensity level
                                    print('made it to beginning of loop')
                                    for contour_path1 in col1.get_paths(): 
                                        # Create the polygon for this intensity level
                                        # The first polygon in the path is the main one, the following ones are "holes"
                                        for ncp1,cp1 in enumerate(contour_path1.to_polygons()):
                                            print(2)
                                            cpa1 = np.asarray(cp1[:])
                                            x1 = cpa1[:,0]
                                            y1 = cpa1[:,1]
                                            new_shape1 = geometry.Polygon([(i[0], i[1]) for i in zip(x1,y1)])
                                            if ncp1 == 0:
                                                poly1 = new_shape1
                                            else:
                                                # Remove the holes if there are any
                                                poly1 = poly1.difference(new_shape)

                                        # do something with polygon
                                        #print(poly.area) 
                                        print(poly1.centroid.x)
                                        s_new1 = transform(proj, poly1)
                                        projected_area1 = (transform(proj, poly1).area * units('m^2')).to('km^2')
                                        if projected_area1 > 5 * units('km^2'):
                                            ref_areas.append((projected_area1))
                                            max_lons_c.append((poly1.centroid.x))
                                            max_lats_c.append((poly1.centroid.y))
                                            print('added polygon')
                                        else:
                                            print('nope')
                            else:
                                ref_areas.append((projected_area))
                                max_lons_c.append((poly.centroid.x))
                                max_lats_c.append((poly.centroid.y))
                                print('added polygon')
                                            
                    #print(s_new)
                max_lons_c = np.asarray(max_lons_c)
                max_lats_c = np.asarray(max_lats_c)
                #ZDRlevels = np.arange(3, np.max(ZDRmasked)+((np.max(ZDRmasked))-3)/2, (np.max(ZDRmasked))-3)
                zdrlev = [1.75]
                kdplev = [.75]
                #ZDRlevels = np.arange(3,5.5,.5)
                #ZDRlevels1 = np.arange(5,10,.5)
                #KDPlevels = np.arange(.75, np.max(KDPmasked)+((np.max(KDPmasked))-1.5)/2, (np.max(KDPmasked))-1.5)
                #KDPlevels = np.arange(.75,1.75,.25)
                #KDPlevels1 = np.arange(1.5,10,.25)
                zdrc = ax.contour(rlons[0,:,:],rlats[0,:,:],ZDRmasked,zdrlev,linewidths = 2, colors='k', alpha = .01)
                #zrdc = ax.contourf(rlons[0,:,:],rlats[0,:,:],ZDRmasked,ZDRlevels,linewide = .01, colors='pink', alpha = .8)
                #ax.contourf(rlons[0,:,:],rlats[0,:,:],ZDRmasked,ZDRlevels1,linewide = .01, colors='crimson', alpha = .8)

                #kdpc = ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels,linewide = .01, colors ='green', alpha = .5)
                kdpc = ax.contour(rlons[0,:,:],rlats[0,:,:],KDPmasked,kdplev,linewidths = 2, colors='green', alpha = .8)
                #ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels1,linewide = .01, colors ='b', alpha = .5)
                print('made it here')
                plt.savefig('testfig.png')

                #proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
                #           pyproj.Proj(init='epsg:3857'))
                zdr_areas = []
                zdr_centroid_lon = []
                zdr_centroid_lat = []
                zdr_mean = []
                zdr_cc_mean = []
                zdr_max = []
                zdr_storm_lon = []
                zdr_storm_lat = []
                zdr_dist = []
                zdr_forw = []
                zdr_back = []
                #print("here too")
                for col in zdrc.collections:
                    # Loop through all polygons that have the same intensity level
                    #print('hi')
                    for contour_path in col.get_paths(): 
                        # Create the polygon for this intensity level
                        # The first polygon in the path is the main one, the following ones are "holes"
                        #print('hi')
                        for ncp,cp in enumerate(contour_path.to_polygons()):
                            #print('hi')
                            cpa = np.asarray(cp[:])
                            x = cpa[:,0]
                            y = cpa[:,1]
                            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
                            if ncp == 0:
                                poly = new_shape
                                #print('hi')
                            else:
                                # Remove the holes if there are any
                                poly = poly.difference(new_shape)
                                #print('hi')

                        # do something with polygon
                        #print(poly.area) 
                        #print(poly.centroid.x)
                        s_new = transform(proj, poly)
                        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
                        boundary = np.asarray(poly.boundary.xy)
                        polypath = Path(boundary.transpose())
                        coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
                        mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                        mean = np.mean(ZDRmasked[mask])
                        mean_cc = np.mean(CC[mask])
                        if projected_area > 1 * units('km^2') and mean > zdrlev[0] and mean_cc > .88:
                            g = Geod(ellps='sphere')
                            dist = np.zeros((np.asarray(max_lons_c).shape[0]))
                            forw = np.zeros((np.asarray(max_lons_c).shape[0]))
                            back = np.zeros((np.asarray(max_lons_c).shape[0]))
                            for i in range(dist.shape[0]):
                                        distance_1 = g.inv(poly.centroid.x, poly.centroid.y,
                                                               max_lons_c[i], max_lats_c[i])
                                        #print(distance_1[2]/1000)
                                        #print(distance_1)
                                        back[i] = distance_1[1]
                                        if distance_1[1] < 0:
                                            back[i] = distance_1[1] + 360
                                        forw[i] = np.abs(back[i] - storm_relative_dir)
                                        dist[i] = distance_1[2]/1000.
                            #print(dist.shape)
                            if (forw[np.where(dist == np.min(dist))[0][0]] < 90 and np.min(dist) < 15.0) or (forw[np.where(dist == np.min(dist))[0][0]] < 140 and np.min(dist) < 6.0):
                                zdr_storm_lon.append((max_lons_c[np.where(dist == np.min(dist))[0][0]]))
                                zdr_storm_lat.append((max_lats_c[np.where(dist == np.min(dist))[0][0]]))
                                zdr_dist.append(np.min(dist))
                                zdr_forw.append(forw[np.where(dist == np.min(dist))[0][0]])
                                zdr_back.append(back[np.where(dist == np.min(dist))[0][0]])
                                zdr_areas.append((projected_area))
                                zdr_centroid_lon.append((poly.centroid.x))
                                zdr_centroid_lat.append((poly.centroid.y))
                                zdr_mean.append((mean))
                                zdr_cc_mean.append((mean_cc))
                                zdr_max.append((np.max(ZDRmasked[mask])))
                                patch = PathPatch(polypath, facecolor='none', alpha=.8, edgecolor = 'blue', linewidth = 3)
                                ax.add_patch(patch)
                        #print(s_new)
                print('made it through zdr centroids')
                
                if len(zdr_storm_lon) > 0:
                    kdp_areas = []
                    kdp_centroid_lon = []
                    kdp_centroid_lat = []
                    kdp_max = []
                    kdp_storm_lon = []
                    kdp_storm_lat = []
                    for col in kdpc.collections:
                        # Loop through all polygons that have the same intensity level
                        for contour_path in col.get_paths(): 
                            # Create the polygon for this intensity level
                            # The first polygon in the path is the main one, the following ones are "holes"
                            for ncp,cp in enumerate(contour_path.to_polygons()):
                                print(1)
                                cpa = np.asarray(cp[:])
                                x = cpa[:,0]
                                y = cpa[:,1]
                                new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
                                if ncp == 0:
                                    poly = new_shape
                                else:
                                    # Remove the holes if there are any
                                    poly = poly.difference(new_shape)

                            # do something with polygon
                            #print(poly.area) 
                            print(poly.centroid.x)
                            s_new = transform(proj, poly)
                            projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
                            boundary = np.asarray(poly.boundary.xy)
                            polypath = Path(boundary.transpose())
                            coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
                            mask_kdp = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                            #mean = np.mean(ZDRmasked[mask])
                            #mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                            #mean = np.mean(REFmasked[mask])
                            if projected_area > 2 * units('km^2'):
                                g = Geod(ellps='sphere')
                                dist_kdp = np.zeros((np.asarray(max_lons_c).shape[0]))
                                for i in range(dist_kdp.shape[0]):
                                            distance_kdp = g.inv(poly.centroid.x, poly.centroid.y,
                                                                   max_lons_c[i], max_lats_c[i])
                                            #print(distance_1[2]/1000)
                                            #print("KDP dist:", distance_kdp)
                                            dist_kdp[i] = distance_kdp[2]/1000.
                                print(dist_kdp)
                                if np.min(np.asarray(dist_kdp)) < 15.0:
                                    #print('Got to KDP stuff')
                                    kdp_areas.append((projected_area))
                                    kdp_centroid_lon.append((poly.centroid.x))
                                    kdp_centroid_lat.append((poly.centroid.y))
                                    kdp_storm_lon.append((max_lons_c[np.where(dist_kdp == np.min(dist_kdp))[0][0]]))
                                    kdp_storm_lat.append((max_lats_c[np.where(dist_kdp == np.min(dist_kdp))[0][0]]))
                                    kdp_max.append((np.max(KDPmasked[mask_kdp])))
                                    patch = PathPatch(polypath, facecolor='none', alpha=.5, edgecolor = 'grey', linewidth = 3)
                                    ax.add_patch(patch)

                    print('made it through kdp centroids')

                    #Consolidating the arc objects associated with each storm:
                    zdr_areas_arr = np.zeros((len(zdr_areas)))
                    zdr_max_arr = np.zeros((len(zdr_max)))
                    zdr_mean_arr = np.zeros((len(zdr_mean)))                    
                    for i in range(len(zdr_areas)):
                        zdr_areas_arr[i] = zdr_areas[i].magnitude
                        zdr_max_arr[i] = zdr_max[i]
                        zdr_mean_arr[i] = zdr_mean[i]

                    zdr_centroid_lons = np.asarray(zdr_centroid_lon)
                    zdr_centroid_lats = np.asarray(zdr_centroid_lat)
                    zdr_con_areas = []
                    zdr_con_maxes = []
                    zdr_con_means = []
                    zdr_con_centroid_lon = []
                    zdr_con_centroid_lat = []
                    zdr_con_max_lon = []
                    zdr_con_max_lat = []

                    #For KDP as well
                    kdp_areas_arr = np.zeros((len(kdp_areas)))
                    kdp_max_arr = np.zeros((len(kdp_max)))
                    for i in range(len(kdp_areas)):
                        kdp_areas_arr[i] = kdp_areas[i].magnitude
                        kdp_max_arr[i] = kdp_max[i]
                    kdp_centroid_lons = np.asarray(kdp_centroid_lon)
                    kdp_centroid_lats = np.asarray(kdp_centroid_lat)
                    kdp_con_areas = []
                    kdp_con_maxes = []
                    kdp_con_centroid_lon = []
                    kdp_con_centroid_lat = []
                    kdp_con_max_lon = []
                    kdp_con_max_lat = []
                    for i in enumerate(zdr_storm_lon):
                        print(i[0])
                        if i[0] != 0:
                            if zdr_storm_lon[i[0]-1] == zdr_storm_lon[i[0]]:
                                #print("Skipping this one")
                                continue
                            else:
                                print(zdr_storm_lon[i[0]])
                                #Find the arc objects associated with this storm:
                                zdr_objects_lons = zdr_centroid_lons[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                zdr_objects_lats = zdr_centroid_lats[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                #print("zdr lons:", zdr_objects_lons)
                                #Get the sum of their areas
                                print(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
                                zdr_con_areas.append(np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                zdr_con_maxes.append(np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                zdr_con_means.append(np.mean(zdr_mean_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                zdr_con_max_lon.append(rlons_2d[np.where(ZDRmasked==np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))])
                                zdr_con_max_lat.append(rlats_2d[np.where(ZDRmasked==np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))])
                                #print("Areas sum:", zdr_con_areas)
                                #Find the actual centroids
                                weighted_lons = zdr_objects_lons * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                zdr_con_centroid_lon.append(np.sum(weighted_lons) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                weighted_lats = zdr_objects_lats * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                zdr_con_centroid_lat.append(np.sum(weighted_lats) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                #print("New centroid lon:", zdr_con_centroid_lon, "New centroid lat:", zdr_con_centroid_lat)
                                #print("lons in loop", zdr_objects_lons)

                                try:
                                    #Find the kdp objects associated with this storm:
                                    kdp_objects_lons = kdp_centroid_lons[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                    #print("kdp lons:", kdp_objects_lons)
                                    kdp_objects_lats = kdp_centroid_lats[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                    #Get the sum of their areas
                                    print(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                                    kdp_con_areas.append(np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                    kdp_con_maxes.append(np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                    kdp_con_max_lon.append(rlons_2d[np.where(KDPmasked==np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))])
                                    kdp_con_max_lat.append(rlats_2d[np.where(KDPmasked==np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))])
                                    #Find the actual centroids
                                    weighted_lons_kdp = kdp_objects_lons * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                    kdp_con_centroid_lon.append(np.sum(weighted_lons_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                    weighted_lats_kdp = kdp_objects_lats * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                    #print("Could be it:","weighted lons:",weighted_lons_kdp, "object lons",kdp_objects_lons, "areas:",kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                                    kdp_con_centroid_lat.append(np.sum(weighted_lats_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                except:
                                    print('storm missing kdp or zdr')
                                    kdp_con_max_lon.append(0)
                                    kdp_con_max_lat.append(0)
                                    kdp_con_centroid_lon.append(0)
                                    kdp_con_centroid_lat.append(0)

                        else:
                            #print(zdr_storm_lon[i[0]])
                            #Find the arc objects associated with this storm:
                            zdr_objects_lons = zdr_centroid_lons[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                            zdr_objects_lats = zdr_centroid_lats[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                            #print("zdr lons:", zdr_objects_lons)
                            #print("arc lats:", zdr_objects_lats)
                            #Get the sum of their areas
                            #print(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
                            zdr_con_areas.append(np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                            zdr_con_maxes.append(np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                            zdr_con_means.append(np.mean(zdr_mean_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                            zdr_con_max_lon.append(rlons_2d[np.where(ZDRmasked==np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))])
                            zdr_con_max_lat.append(rlats_2d[np.where(ZDRmasked==np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))])
                            #print("Areas sum:",zdr_con_areas)
                            #Find the actual centroids
                            weighted_lons = zdr_objects_lons * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                            zdr_con_centroid_lon.append(np.sum(weighted_lons) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                            weighted_lats = zdr_objects_lats * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                            zdr_con_centroid_lat.append(np.sum(weighted_lats) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                            #print("New centroid lon:", zdr_con_centroid_lon, "New centroid lat:", zdr_con_centroid_lat)
                            #print("lons out of loop", zdr_objects_lons)
                            try:
                                #Find the kdp objects associated with this storm:
                                kdp_objects_lons = kdp_centroid_lons[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                #print("kdp lons:", kdp_objects_lons)
                                kdp_objects_lats = kdp_centroid_lats[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                #Get the sum of their areas
                                #print(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                                kdp_con_areas.append(np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                kdp_con_maxes.append(np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                kdp_con_max_lon.append(rlons_2d[np.where(KDPmasked==np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))])
                                kdp_con_max_lat.append(rlats_2d[np.where(KDPmasked==np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))])
                                #Find the actual centroids
                                weighted_lons_kdp = kdp_objects_lons * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                kdp_con_centroid_lon.append(np.sum(weighted_lons_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                weighted_lats_kdp = kdp_objects_lats * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                #print("Could be it:","weighted lons:",weighted_lons_kdp, "object lons",kdp_objects_lons, "areas:",kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                                kdp_con_centroid_lat.append(np.sum(weighted_lats_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                            except:
                                print('storm missing kdp or zdr')
                                kdp_con_max_lon.append(0)
                                kdp_con_max_lat.append(0)
                                kdp_con_centroid_lon.append(0)
                                kdp_con_centroid_lat.append(0)
                    
                    #Calculate KDP-ZDR separation
                    print('calculating separation')
                    kdp_con_centroid_lons1 = np.asarray(kdp_con_centroid_lon)
                    kdp_con_centroid_lats1 = np.asarray(kdp_con_centroid_lat)
                    zdr_con_centroid_lons1 = np.asarray(zdr_con_centroid_lon)
                    zdr_con_centroid_lats1 = np.asarray(zdr_con_centroid_lat)
                    #Eliminate consolidated arcs smaller than 10km
                    area = 1.5
                    zdr_con_areas_arr = np.asarray(zdr_con_areas)
                    zdr_con_centroid_lats = zdr_con_centroid_lats1[zdr_con_areas_arr > area]
                    zdr_con_centroid_lons = zdr_con_centroid_lons1[zdr_con_areas_arr > area]
                    kdp_con_centroid_lats = kdp_con_centroid_lats1[zdr_con_areas_arr > area]
                    kdp_con_centroid_lons = kdp_con_centroid_lons1[zdr_con_areas_arr > area]
                    zdr_con_max_lons1 = np.asarray(zdr_con_max_lon)[zdr_con_areas_arr > area]
                    zdr_con_max_lats1 = np.asarray(zdr_con_max_lat)[zdr_con_areas_arr > area]
                    kdp_con_max_lons1 = np.asarray(kdp_con_max_lon)[zdr_con_areas_arr > area]
                    kdp_con_max_lats1 = np.asarray(kdp_con_max_lat)[zdr_con_areas_arr > area]
                    print('made it past reset')
                    zdr_con_areas1 = zdr_con_areas_arr[zdr_con_areas_arr > area]

                    kdp_inds = np.where(kdp_con_centroid_lats > 0)
                    distance_kdp_zdr = g.inv(kdp_con_centroid_lons[kdp_inds], kdp_con_centroid_lats[kdp_inds], zdr_con_centroid_lons[kdp_inds], zdr_con_centroid_lats[kdp_inds])
                    dist_kdp_zdr = distance_kdp_zdr[2] / 1000.
                    #Now make an array for the distances which will have the same shape as the lats to prevent errors
                    shaped_dist = np.zeros((np.shape(kdp_con_centroid_lats)))
                    shaped_dist[kdp_inds] = dist_kdp_zdr
                    print('maybe its here')
                    #Do the same for the distances between the maxes
                    distance_kdp_zdr_max = g.inv(kdp_con_max_lons1[kdp_inds], kdp_con_max_lats1[kdp_inds], zdr_con_max_lons1[kdp_inds], zdr_con_max_lats1[kdp_inds])
                    dist_kdp_zdr_max = distance_kdp_zdr_max[2] / 1000.
                    #Now make an array for the distances which will have the same shape as the lats to prevent errors
                    shaped_dist_max = np.zeros((np.shape(kdp_con_centroid_lats)))
                    shaped_dist_max[kdp_inds] = dist_kdp_zdr_max
                    print('or not')
                    
                else:
                    kdp_areas = []
                    kdp_centroid_lon = []
                    kdp_centroid_lat = []
                    kdp_storm_lon = []
                    kdp_storm_lat = []
                    zdr_con_centroid_lats = []
                    zdr_con_centroid_lons = []
                    kdp_con_centroid_lats = []
                    kdp_con_centroid_lons = []
                    kdp_con_area = []
                    zdr_con_areas1 = []
                    
                #Now start plotting stuff!
                print('made it through giant if statement')
                if np.asarray(zdr_centroid_lon).shape[0] > 0:
                    ax.scatter(zdr_centroid_lon, zdr_centroid_lat, marker = '*', s = 100, color = 'black', zorder = 10, transform=ccrs.PlateCarree())
                    ax.scatter(zdr_con_max_lon, zdr_con_max_lat, marker = '*', s = 100, color = 'purple', zorder = 10, transform=ccrs.PlateCarree())
                if np.asarray(kdp_centroid_lon).shape[0] > 0:
                    ax.scatter(kdp_centroid_lon, kdp_centroid_lat, marker = '^', s = 100, color = 'black', zorder = 10, transform=ccrs.PlateCarree())
                if np.asarray(kdp_centroid_lon).shape[0] > 0:    
                    ax.scatter(kdp_con_max_lon, kdp_con_max_lat, marker = '^', s = 100, color = 'purple', zorder = 10, transform=ccrs.PlateCarree())
                print("plotted centroids")
                #Uncomment to print all object areas
                #for i in enumerate(zdr_areas):
                #    plt.text(zdr_centroid_lon[i[0]]+.016, zdr_centroid_lat[i[0]]+.016, "%.2f km^2" %(zdr_areas[i[0]].magnitude), size = 23)
                    #plt.text(zdr_centroid_lon[i[0]]+.016, zdr_centroid_lat[i[0]]+.016, "%.2f km^2 / %.2f km / %.2f dB" %(zdr_areas[i[0]].magnitude, zdr_dist[i[0]], zdr_forw[i[0]]), size = 23)
                    #plt.annotate(zdr_areas[i[0]], (zdr_centroid_lon[i[0]],zdr_centroid_lat[i[0]]))
                #ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels1,linewide = .01, colors ='b', alpha = .5)
                #plt.tight_layout()
                #plt.savefig('ZDRarcannotated.png')
                
                #Plot the consolidated stuff!
                if len(zdr_con_areas1) > 0:
                    #ax.scatter(zdr_con_centroid_lon, zdr_con_centroid_lat, marker = '*', s = 500, color = 'orange', zorder = 10, transform=ccrs.PlateCarree())
                    try:
                        for i in enumerate(zdr_con_centroid_lats):
                            print("consolidated ZDR:")
                            ax.scatter(zdr_con_centroid_lons, zdr_con_centroid_lats, marker = '*', s = 500, color = 'orange', zorder = 10, transform=ccrs.PlateCarree())
                            try:
                                plt.text(zdr_con_centroid_lons[i[0]]+.025, zdr_con_centroid_lats[i[0]]+.016, "%.2f km^2 / %.2f dB" %(zdr_con_areas1[i[0]], zdr_con_maxes[i[0]]), size = 23)
                            except:
                                print("oops zdr")
                        #plt.text(kdp_con_centroid_lon[i[0]]-.20, kdp_con_centroid_lat[i[0]]+.016, "%.2f km" %(dist_kdp_zdr[i[0]]), size = 23, color = 'red')                
                    except:
                        print('failed')
                        try:
                            plt.text(float(zdr_con_centroid_lons)+.016, float(zdr_con_centroid_lats)+.016, "%.2f km^2 / %.2f dB" %(zdr_con_areas1[i[0]], zdr_con_maxes[i[0]]), size = 23)
                        except:
                            print('no zdr centroids')
                        #plt.text(float(kdp_con_centroid_lon)-.20, float(kdp_con_centroid_lat)+.016, "%.2f km" %(float(dist_kdp_zdr[0])), size = 23, color = 'red')
                if len(kdp_con_areas) > 0:
                    #ax.scatter(zdr_con_centroid_lon, zdr_con_centroid_lat, marker = '*', s = 500, color = 'orange', zorder = 10, transform=ccrs.PlateCarree())
                    try:
                        for i in kdp_inds[0]:
                            #plt.text(zdr_con_centroid_lon[i[0]]+.025, zdr_con_centroid_lat[i[0]]+.016, "%.2f km^2" %(zdr_con_areas[i[0]].magnitude), size = 23)
                            try:
                                plt.text(kdp_con_centroid_lons[i]-.20, kdp_con_centroid_lats[i]+.016, "%.2f km / %.2f km" %(shaped_dist[i], shaped_dist_max[i]), size = 23, color = 'red')                
                            except:
                                print('oops kdp')
                    except:
                        print('failed')
                        #plt.text(float(zdr_con_centroid_lon)+.016, float(zdr_con_centroid_lat)+.016, "%.2f km^2" %(float(zdr_con_areas[0])), size = 23)
                        try:
                            plt.text(float(kdp_con_centroid_lons)-.20, float(kdp_con_centroid_lats)+.016, "%.2f km / %.2f km" %(float(shaped_dist[0]), float(shaped_dist_max[0])), size = 23, color = 'red')
                        except:
                            print('no kdp centroids')
                print("means there's a kdp problem")
                hour = time_start.hour
                if hour < 10:
                    hour = '0'+str(hour)
                minute = time_start.minute
                if minute < 10:
                    minute = '0'+str(minute)
                day = time_start.day
                if day < 10:
                    day = '0'+str(day)
                title_plot = plt.title(station+' Radar Reflectivity, ZDR, and KDP '+str(time_start.year)+'-'+str(time_start.month)+'-'+str(time_start.day)+
                                           ' '+str(hour)+':'+str(minute)+' UTC', size = 25)
                #if np.asarray(zdr_storm_lon).shape[0] > 0:
                #    ax.scatter(zdr_storm_lon,zdr_storm_lat, marker = "o", color = 'purple', s = 500)
                #if np.asarray(kdp_storm_lon).shape[0] > 0:
                #    ax.scatter(kdp_storm_lon,kdp_storm_lat, marker = "o", color = 'purple', s = 500)
                try:
                    ax.scatter(max_lons_c,max_lats_c, marker = "o", color = 'k', s = 500, alpha = .6)
                except:
                    "No storm centroids found"
                try:
                    plt.plot([zdr_con_centroid_lons[kdp_inds], kdp_con_centroid_lons[kdp_inds]], [zdr_con_centroid_lats[kdp_inds],kdp_con_centroid_lats[kdp_inds]], color = 'red', linewidth = 5, transform=ccrs.PlateCarree())
                except:
                    print('KDP-ZDR separation didt work')
                alt_levs = [1000, 2000]
                cele = ax.contour(ungrid_lons,ungrid_lats,gate_altitude,alt_levs, linewidths = 7, alpha = .6, colors = 'grey')
                plt.clabel(cele, fontsize=18, inline=1, inline_spacing=10, fmt='%i', rightside_up=True, use_clabeltext=True)
                plt.tight_layout()
                plt.savefig('REFt_CCtropic_ZDRArc_comp'+station+str(time_start.year)+str(time_start.month)+str(day)+str(hour)+str(minute)+'.png')
                print('figure saved')
                plt.close()
                n = n+1
    except:
        continue

plt.show()

in loop
17
0.449425
2017-10-08 00:01:55.705000
its this line
heres the problem
almost gridding
made it to smoothing
made it through gradient
got gradient
Set up our projection
(-87.769722222222228, 32.172222222222217, <cartopy.crs.PlateCarree object at 0x000001786D72FBA0>)
plotting
1
-86.69065226444542
past mask
43.4372
found a storm
found a big storm
plotted a big storm
made it to beginning of loop
2
-86.64611739455415
added polygon
2
-86.68603101581174
added polygon
1
-86.88844074584246
past mask
40.6789
found a storm
added polygon
1
-87.52206603970714
past mask
40.1789
1
-87.06800999334027
past mask
41.1452
found a storm
added polygon
1
-87.24191648348776
past mask
41.9684
found a storm
added polygon
1
-87.92910912117509
past mask
42.9573
found a storm
found a big storm
plotted a big storm
made it to beginning of loop
2
-87.91201954420984
added polygon
made it here
made it through zdr centroids
1
-86.59676765476696
[   4.95185999   14.80658819   27.64555321   46.2393998    64.373501

C:\Users\mwilson41\AppData\Local\Continuum\Anaconda3\envs\radar\lib\site-packages\ipykernel_launcher.py:559: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


figure saved
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
0.448452
2017-10-08 00:06:41.239000
its this line
heres the problem
almost gridding
made it to smoothing
made it through gradient
got gradient
Set up our projection
(-87.769722222222228, 32.172222222222217, <cartopy.crs.PlateCarree object at 0x000001786D7870A0>)
plotting
1
-86.69959896954948
past mask
43.4465
found a storm
found a big storm
plotted a big storm
made it to beginning of loop
2
-86.61134325930838
added polygon
2
-86.71383601533819
added polygon
1
-86.90820224539154
past mask
40.3184
1
-87.0845090136728
past mask
41.3592
found a storm
added polygon
1
-87.25227799493692
past mask
41.8872
found a storm
added polygon
1
-87.94394289681152
past mask
42.7311
found a storm
found a big storm
plotted a big storm
made it to beginning of loop
2
-87.92827465446022
added polygon
1
-87.35021

C:\Users\mwilson41\AppData\Local\Continuum\Anaconda3\envs\radar\lib\site-packages\ipykernel_launcher.py:559: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


figure saved
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
in loop
17
0.447338
2017-10-08 00:16:11.832000
its this line
heres the problem
almost gridding
made it to smoothing
made it through gradient
got gradient
Set up our projection
(-87.769722222222228, 32.172222222222217, <cartopy.crs.PlateCarree object at 0x000001780CA1F308>)
plotting
1
-88.04420035722877
past mask
41.1243
found a storm
added polygon
1
-86.72637568057617
past mask
43.7411
found a storm
found a big storm
plotted a big storm
made it to beginning of loop
2
-86.65834565753897
added polygon
2
-86.7691516489436
added polygon
1
-86.97064315272648
past mask
40.573
found a storm
added polygon
1
-87.10782202079874
past mask
42.7652
found a storm
added polygon
1
-87.27838645188457
past mask
40.6694
found a storm
added polygon
1
-87.95958536428967
past mask
42.5048
found a storm
added polygon
1
-87

...

figure saved
in loop
23
in loop
23
in loop
23
in loop
23
in loop
23
0.404278
2017-10-08 03:26:53.095000
its this line
heres the problem
almost gridding
made it to smoothing
made it through gradient
got gradient
Set up our projection
(-87.769722222222228, 32.172222222222217, <cartopy.crs.PlateCarree object at 0x000001780CA602B0>)
plotting
1
-87.02863121500965
past mask
40.9226
found a storm
added polygon
1
-87.027624160069
past mask
42.4458
found a storm
added polygon
1
-87.1149342132193
past mask
41.0552
found a storm
added polygon
1
-87.88245557725021
past mask
40.4603
found a storm
added polygon
1
-87.9057771138719
past mask
40.2847
found a storm
added polygon
1
-87.16700233610054
past mask
40.0688
1
-87.07458801336976
past mask
42.5281
found a storm
found a big storm
plotted a big storm
made it to beginning of loop
2
-87.09159458958001
added polygon
1
-87.06240230823629
past mask
39.7349
made it here
made it through zdr centroids
1
-86.98449439139787
[ 46.59719972   8.77955938  23.5

C:\Users\mwilson41\AppData\Local\Continuum\Anaconda3\envs\radar\lib\site-packages\ipykernel_launcher.py:559: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


figure saved
in loop
23
in loop
23
in loop
23
in loop
23
in loop
23
in loop
23
0.448223
2017-10-08 03:28:29.782000
its this line
heres the problem
almost gridding
made it to smoothing
made it through gradient
got gradient
Set up our projection
(-87.769722222222228, 32.172222222222217, <cartopy.crs.PlateCarree object at 0x000001780488E938>)
plotting
1
-87.03154811803904
past mask
40.923
found a storm
added polygon
1
-87.0319055111365
past mask
42.4162
found a storm
added polygon
1
-87.1159091525399
past mask
40.6968
found a storm
added polygon
1
-87.89436127640387
past mask
40.1468
found a storm
added polygon
1
-87.90938361334214
past mask
40.2025
found a storm
added polygon
1
-87.0778773455124
past mask
42.5233
found a storm
found a big storm
plotted a big storm
made it to beginning of loop
2
-87.09653331748372
added polygon
1
-87.05680854265277
past mask
39.635
made it here
made it through zdr centroids
1
-87.0410494348911
[ 47.74188977   6.24308947  19.49946221  83.43939638  86.84647

C:\Users\mwilson41\AppData\Local\Continuum\Anaconda3\envs\radar\lib\site-packages\ipykernel_launcher.py:559: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


figure saved
in loop
23
in loop
23
in loop
23
in loop
23
in loop
23


In [ ]:
#kdp_con_max_lats1 = np.asarray(kdp_con_max_lat)[zdr_con_areas_arr > area]
print(kdp_con_centroid_lat)
print(zdr_con_centroid_lat)

In [ ]:
print(zdr_con_max_lon[:])
print(kdp_con_max_lon)
print(kdp_con_centroid_lons1)
print(zdr_con_centroid_lons)
print(kdp_con_max_lats1)
print(kdp_con_centroid_lons1)
print(shaped_dist)
print(zdr_con_max_lon)
print(kdp_con_centroid_lons1)

In [ ]:
zdr_con_max_lons1 = np.asarray(zdr_con_max_lon)[zdr_con_areas_arr > area]

In [ ]:
print(np.asarray(zdr_con_max_lon)[zdr_con_areas_arr > area])

In [ ]:
print(zdr_mean)
print(zdr_cc_mean)
print(zdr_con_maxes)
rlons_2 = rlons[0,:,:]
rlats_2 = rlats[0,:,:]
print(np.where(ZDRmasked==zdr_con_maxes))
print(rlons_2[np.where(ZDRmasked==zdr_con_maxes)])
print(rlats_2[np.where(ZDRmasked==zdr_con_maxes)])
zdr_con_max_lon = rlons_2[np.where(ZDRmasked==zdr_con_maxes)]
zdr_con_max_lat = rlats_2[np.where(ZDRmasked==zdr_con_maxes)]


In [ ]:
print(zdr_con_centroid_lats.shape)
print(kdp_con_centroid_lats.shape)
print(shaped_dist.shape)
print(np.asarray(zdr_con_areas).shape)
for i in enumerate(zdr_con_areas):
        print("consolidated ZDR:")
        print(zdr_con_centroid_lon[i[0]]+.025, zdr_con_centroid_lat[i[0]]+.016, zdr_con_areas[i[0]])
print(zdr_con_areas)

zdr_con_areas_arr = np.asarray(zdr_con_areas)
zdr_con_centroid_lats1 = zdr_con_centroid_lats[zdr_con_areas_arr > 10]
zdr_con_centroid_lons1 = zdr_con_centroid_lons[zdr_con_areas_arr > 10]
kdp_con_centroid_lats1 = kdp_con_centroid_lats[zdr_con_areas_arr > 10]
kdp_con_centroid_lons1 = kdp_con_centroid_lons[zdr_con_areas_arr > 10]
shaped_dist1 = shaped_dist[zdr_con_areas_arr > 10]
zdr_con_areas1 = zdr_con_areas_arr[zdr_con_areas_arr > 10]

In [ ]:
#print(max_lons_c)
#print(max_lats_c)
#print(radar.fields)
print(gate_altitude)

In [ ]:
print(shaped_dist)
print(kdp_con_centroid_lats)
print(kdp_inds)
for i in kdp_inds[0]:
    print(shaped_dist[i])
    print(kdp_con_centroid_lats[i])

In [ ]:
REFlev = [40]
refc = ax.contour(rlons[0,:,:],rlats[0,:,:],smoothed_ref,REFlev)

ref_areas = []
max_lons_c = []
max_lats_c = []

#Look for reflectivity centroids
for col in refc.collections:
    # Loop through all polygons that have the same intensity level
    for contour_path in col.get_paths(): 
        # Create the polygon for this intensity level
        # The first polygon in the path is the main one, the following ones are "holes"
        for ncp,cp in enumerate(contour_path.to_polygons()):
            print(1)
            cpa = np.asarray(cp[:])
            x = cpa[:,0]
            y = cpa[:,1]
            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
            if ncp == 0:
                poly = new_shape
            else:
                # Remove the holes if there are any
                poly = poly.difference(new_shape)

        # do something with polygon
        #print(poly.area) 
        print(poly.centroid.x)
        s_new = transform(proj, poly)
        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
        if projected_area > 20 * units('km^2'):
            ref_areas.append((projected_area))
            max_lons_c.append((poly.centroid.x))
            max_lats_c.append((poly.centroid.y))
            print('added polygon')

In [ ]:
print(len(zdr_con_areas))
print(zdr_con_areas)
#print(dist_kdp_zdr)
print(zdr_con_centroid_lat)
print(kdp_inds)
print(kdp_con_centroid_lats[kdp_inds])
print(kdp_con_centroid_lats)
print(kdp_con_centroid_lat)
print(kdp_con_centroid_lats[kdp_inds])

In [ ]:
fig=plt.figure(n,figsize=(30.,25.))
ax = plt.subplot(111,projection=ccrs.PlateCarree())
ax.coastlines('50m',edgecolor='black',linewidth=0.75)
#ax.add_feature(states_provinces,edgecolor='black',linewidth=0.5)
ax.add_feature(counties, edgecolor = 'black', linewidth = 0.5)
ax.add_feature(states, edgecolor = 'black', linewidth = 1.5)
ax.set_extent([LL[0],UR[0],LL[1],UR[1]])
# It is used within peak_local_max function
smoothed_ref = ndi.gaussian_filter(REFmasked, sigma = 3, order = 0)
image_max = ndi.maximum_filter(smoothed_ref, size=15, mode='constant')

# Comparison between image_max and im to find the coordinates of local maxima
coordinates = peak_local_max(smoothed_ref, min_distance=20)
ref_maxes = REFmasked[coordinates[:,0], coordinates[:,1]]
max_lons = rlons[0,coordinates[:,0], coordinates[:,1]]
max_lats = rlats[0,coordinates[:,0], coordinates[:,1]]
#max_lons_c = max_lons[ref_maxes > 45]
#max_lats_c = max_lats[ref_maxes > 45]
alt_levs = [2000]
ax.contour(ungrid_lons,ungrid_lats,gate_altitude,alt_levs, linewidths = 7)
REFlevels = np.arange(20,73,2)
REFlev = [40]
ax.contourf(rlons[0,:,:],rlats[0,:,:],smoothed_ref,REFlevels,cmap = plt.cm.gist_ncar)
REFlev = [40]
refc = ax.contour(rlons[0,:,:],rlats[0,:,:],smoothed_ref,REFlev)

ref_areas = []
max_lons_c = []
max_lats_c = []

#Look for reflectivity centroids
for col in refc.collections:
    # Loop through all polygons that have the same intensity level
    for contour_path in col.get_paths(): 
        # Create the polygon for this intensity level
        # The first polygon in the path is the main one, the following ones are "holes"
        for ncp,cp in enumerate(contour_path.to_polygons()):
            print(1)
            cpa = np.asarray(cp[:])
            x = cpa[:,0]
            y = cpa[:,1]
            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
            if ncp == 0:
                poly = new_shape
            else:
                # Remove the holes if there are any
                poly = poly.difference(new_shape)

        # do something with polygon
        #print(poly.area) 
        print(poly.centroid.x)
        s_new = transform(proj, poly)
        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
        if projected_area > 20 * units('km^2'):
            ref_areas.append((projected_area))
            max_lons_c.append((poly.centroid.x))
            max_lats_c.append((poly.centroid.y))

    #print(s_new)

#Uncomment to print all object areas
#for i in enumerate(zdr_areas):
#    plt.text(zdr_centroid_lon[i[0]]+.016, zdr_centroid_lat[i[0]]+.016, "%.2f km^2" %(zdr_areas[i[0]].magnitude), size = 23)
    #plt.text(zdr_centroid_lon[i[0]]+.016, zdr_centroid_lat[i[0]]+.016, "%.2f km^2 / %.2f km / %.2f dB" %(zdr_areas[i[0]].magnitude, zdr_dist[i[0]], zdr_forw[i[0]]), size = 23)
    #plt.annotate(zdr_areas[i[0]], (zdr_centroid_lon[i[0]],zdr_centroid_lat[i[0]]))
#ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels1,linewide = .01, colors ='b', alpha = .5)
#plt.tight_layout()
#plt.savefig('ZDRarcannotated.png')


hour = time_start.hour
if hour < 10:
    hour = '0'+str(hour)
minute = time_start.minute
if minute < 10:
    minute = '0'+str(minute)
day = time_start.day
if day < 10:
    day = '0'+str(day)
title_plot = plt.title(station+' Radar Reflectivity, ZDR, and KDP '+str(time_start.year)+'-'+str(time_start.month)+'-'+str(time_start.day)+
                           ' '+str(hour)+':'+str(minute)+' UTC', size = 25)
#if np.asarray(zdr_storm_lon).shape[0] > 0:
#    ax.scatter(zdr_storm_lon,zdr_storm_lat, marker = "o", color = 'purple', s = 500)
#if np.asarray(kdp_storm_lon).shape[0] > 0:
#    ax.scatter(kdp_storm_lon,kdp_storm_lat, marker = "o", color = 'purple', s = 500)
ax.scatter(max_lons_c,max_lats_c, marker = "o", color = 'k', s = 500, alpha = .6)
try:
    plt.plot([zdr_con_centroid_lons[kdp_inds], kdp_con_centroid_lons[kdp_inds]], [zdr_con_centroid_lats[kdp_inds],kdp_con_centroid_lats[kdp_inds]], color = 'red', linewidth = 5, transform=ccrs.PlateCarree())
except:
    print('KDP-ZDR separation didt work')
#plt.savefig('FFD_ZDRArc_comp'+station+str(time_start.year)+str(time_start.month)+str(day)+str(hour)+str(minute)+'.png')
print('figure saved')
plt.show()

In [ ]:
print(len(kdp_con_areas))

In [ ]:
#THIS CELL HAS THE PROBLEM MAKE DIST_KDP THE SAME SHAPE AS EVERYTHING ELSE

kdp_inds = np.where(kdp_con_centroid_lats > 0)
print(kdp_inds)
print(kdp_con_centroid_lon)
print(dist_kdp_zdr)
shaped_dist = np.zeros((np.shape(kdp_con_centroid_lats)))
print(shaped_dist.shape)
shaped_dist[kdp_inds] = dist_kdp_zdr
for i in kdp_inds:
    print(i)
    print(kdp_con_centroid_lon[i[0]])
    print(kdp_con_centroid_lat[i[0]])
    print(shaped_dist[i[0]])

In [ ]:
#Calculate KDP-ZDR separation
kdp_con_centroid_lons = np.asarray(kdp_con_centroid_lon)
kdp_con_centroid_lats = np.asarray(kdp_con_centroid_lat)
zdr_con_centroid_lons = np.asarray(zdr_con_centroid_lon)
zdr_con_centroid_lats = np.asarray(zdr_con_centroid_lat)
distance_kdp_zdr = g.inv(kdp_con_centroid_lons[kdp_inds], kdp_con_centroid_lats[kdp_inds], zdr_con_centroid_lons[kdp_inds], zdr_con_centroid_lats[kdp_inds])
dist_kdp_zdr = distance_kdp_zdr[2] / 1000.

In [ ]:
kdp_inds = np.where(kdp_con_centroid_lats > 0)
print(kdp_inds[0])

In [ ]:
print(zdr_storm_lon)

In [ ]:
for i in enumerate(zdr_storm_lon):
    print(i[0])
    if i[0] != 0:
        if zdr_storm_lon[i[0]-1] == zdr_storm_lon[i[0]]:
            print("Skipping this one")
            #print("last lon:",zdr_storm_lon[i[0]-1]
            #print("this lon:",zdr_storm_lon[i[0]])

            continue
        else:
            print("using lon:",zdr_storm_lon[i[0]])

In [ ]:
zdr_areas_arr = np.zeros((len(zdr_areas)))
for i in range(len(zdr_areas)):
    zdr_areas_arr[i] = zdr_areas[i].magnitude
zdr_centroid_lons = np.asarray(zdr_centroid_lon)
zdr_centroid_lats = np.asarray(zdr_centroid_lat)
zdr_con_areas = []
zdr_con_centroid_lon = []
zdr_con_centroid_lat = []

#For KDP as well
kdp_areas_arr = np.zeros((len(kdp_areas)))
for i in range(len(kdp_areas)):
    kdp_areas_arr[i] = kdp_areas[i].magnitude
kdp_centroid_lons = np.asarray(kdp_centroid_lon)
kdp_centroid_lats = np.asarray(kdp_centroid_lat)
kdp_con_areas = []
kdp_con_centroid_lon = []
kdp_con_centroid_lat = []
dist_kdp_zdr = []
for i in enumerate(zdr_storm_lon):
    print(i[0])
    if i[0] != 0:
        if zdr_storm_lon[i[0]-1] == zdr_storm_lon[i[0]]:
            print("Skipping this one")
            continue
        else:
            print(zdr_storm_lon[i[0]])
            #Find the arc objects associated with this storm:
            zdr_objects_lons = zdr_centroid_lons[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
            zdr_objects_lats = zdr_centroid_lats[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
            print("zdr lons:", zdr_objects_lons)
            #Get the sum of their areas
            print(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
            zdr_con_areas.append(np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
            print("Areas sum:", zdr_con_areas)
            #Find the actual centroids
            weighted_lons = zdr_objects_lons * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
            zdr_con_centroid_lon.append(np.sum(weighted_lons) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
            weighted_lats = zdr_objects_lats * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
            zdr_con_centroid_lat.append(np.sum(weighted_lats) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
            print("New centroid lon:", zdr_con_centroid_lon, "New centroid lat:", zdr_con_centroid_lat)
            print("lons in loop", zdr_objects_lons)

            try:
                #Find the kdp objects associated with this storm:
                kdp_objects_lons = kdp_centroid_lons[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                print("kdp lons:", kdp_objects_lons)
                kdp_objects_lats = kdp_centroid_lats[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                #Get the sum of their areas
                print(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                kdp_con_areas.append(np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                #Find the actual centroids
                weighted_lons_kdp = kdp_objects_lons * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                kdp_con_centroid_lon.append(np.sum(weighted_lons_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                weighted_lats_kdp = kdp_objects_lats * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                print("Could be it:","weighted lons:",weighted_lons_kdp, "object lons",kdp_objects_lons, "areas:",kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                kdp_con_centroid_lat.append(np.sum(weighted_lats_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                #Get the distance between the KDP and ZDR centroids
                print(kdp_con_centroid_lon, kdp_con_centroid_lat, zdr_con_centroid_lon, zdr_con_centroid_lat)
                distance_kdp_zdr = g.inv(kdp_con_centroid_lon, kdp_con_centroid_lat, zdr_con_centroid_lon, zdr_con_centroid_lat)
                #print(distance_1[2]/1000)
                print("separation:", distance_kdp_zdr[2])
                dist_kdp_zdr.append(distance_kdp_zdr[2]/1000.)
            except:
                print('storm missing kdp or zdr')
    else:
        print(zdr_storm_lon[i[0]])
        #Find the arc objects associated with this storm:
        zdr_objects_lons = zdr_centroid_lons[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
        zdr_objects_lats = zdr_centroid_lats[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
        print("zdr lons:", zdr_objects_lons)
        print("arc lats:", zdr_objects_lats)
        #Get the sum of their areas
        print(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
        zdr_con_areas.append(np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
        print("Areas sum:",zdr_con_areas)
        #Find the actual centroids
        weighted_lons = zdr_objects_lons * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
        zdr_con_centroid_lon.append(np.sum(weighted_lons) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
        weighted_lats = zdr_objects_lats * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
        zdr_con_centroid_lat.append(np.sum(weighted_lats) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
        print("New centroid lon:", zdr_con_centroid_lon, "New centroid lat:", zdr_con_centroid_lat)
        print("lons out of loop", zdr_objects_lons)
        try:
            #Find the kdp objects associated with this storm:
            kdp_objects_lons = kdp_centroid_lons[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
            print("kdp lons:", kdp_objects_lons)
            kdp_objects_lats = kdp_centroid_lats[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
            #Get the sum of their areas
            print(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
            kdp_con_areas.append(np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
            #Find the actual centroids
            weighted_lons_kdp = kdp_objects_lons * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
            kdp_con_centroid_lon.append(np.sum(weighted_lons_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
            weighted_lats_kdp = kdp_objects_lats * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
            print("Could be it:","weighted lons:",weighted_lons_kdp, "object lons",kdp_objects_lons, "areas:",kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
            kdp_con_centroid_lat.append(np.sum(weighted_lats_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
            #Get the distance between the KDP and ZDR centroids
            print(kdp_con_centroid_lon, kdp_con_centroid_lat, zdr_con_centroid_lon, zdr_con_centroid_lat)
            distance_kdp_zdr = g.inv(kdp_con_centroid_lon, kdp_con_centroid_lat, zdr_con_centroid_lon, zdr_con_centroid_lat)
            #print(distance_1[2]/1000)
            print("separation:", distance_kdp_zdr[2])
            dist_kdp_zdr.append(distance_kdp_zdr[2]/1000.)
        except:
            print('storm missing kdp or zdr')
                

In [ ]:
kdp_con_centroid_lons = np.asarray(kdp_con_centroid_lon)
kdp_con_centroid_lats = np.asarray(kdp_con_centroid_lat)
zdr_con_centroid_lons = np.asarray(zdr_con_centroid_lon)
zdr_con_centroid_lats = np.asarray(zdr_con_centroid_lat)
kdp_inds = 
distance_kdp_zdr = g.inv(kdp_con_centroid_lons[kdp_inds], kdp_con_centroid_lats[kdp_inds], zdr_con_centroid_lons[kdp_inds], zdr_con_centroid_lats[kdp_inds])
dist_kdp_zdr = distance_kdp_zdr[2] / 1000.
print(dist_kdp_zdr)

In [ ]:
print(zdr_storm_lon)
print(kdp_storm_lon)
print(zdr_areas)
zdr_areas_arr = np.zeros((len(zdr_areas)))
for i in range(len(zdr_areas)):
    zdr_areas_arr[i] = zdr_areas[i].magnitude
zdr_centroid_lons = np.asarray(zdr_centroid_lon)
zdr_centroid_lats = np.asarray(zdr_centroid_lat)
zdr_con_areas = []
zdr_con_centroid_lon = []
zdr_con_centroid_lat = []

#For KDP as well
kdp_areas_arr = np.zeros((len(kdp_areas)))
for i in range(len(kdp_areas)):
    kdp_areas_arr[i] = kdp_areas[i].magnitude
kdp_centroid_lons = np.asarray(kdp_centroid_lon)
kdp_centroid_lats = np.asarray(kdp_centroid_lat)
kdp_con_areas = []
kdp_con_centroid_lon = []
kdp_con_centroid_lat = []
dist_kdp_zdr = []
for i in enumerate(zdr_storm_lon):
    print(i[0])
    if i[0] != 0:
        if zdr_storm_lon[i[0]-1] == zdr_storm_lon[i[0]]:
            print("Skipping this one")
            continue
        else:
            print(zdr_storm_lon[i[0]])
            #Find the arc objects associated with this storm:
            zdr_objects_lons = zdr_centroid_lons[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
            zdr_objects_lats = zdr_centroid_lats[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
            print("zdr lons:", zdr_objects_lons)
            #Get the sum of their areas
            print(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
            zdr_con_areas.append(np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
            print("Areas sum:", zdr_con_areas)
            #Find the actual centroids
            weighted_lons = zdr_objects_lons * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
            zdr_con_centroid_lon = np.sum(weighted_lons) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
            weighted_lats = zdr_objects_lats * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
            zdr_con_centroid_lat = np.sum(weighted_lats) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
            print("New centroid lon:", zdr_con_centroid_lon, "New centroid lat:", zdr_con_centroid_lat)
            print("lons in loop", zdr_objects_lons)
            
            try:
                #Find the kdp objects associated with this storm:
                kdp_objects_lons = kdp_centroid_lons[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                print("kdp lons:", kdp_objects_lons)
                kdp_objects_lats = kdp_centroid_lats[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                #Find the actual centroids
                weighted_lons_kdp = kdp_objects_lons * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                kdp_con_centroid_lon = np.sum(weighted_lons_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                weighted_lats_kdp = kdp_objects_lats * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                print("Could be it:","weighted lons:",weighted_lons_kdp, "object lons",kdp_objects_lons, "areas:",kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                kdp_con_centroid_lat = np.sum(weighted_lats_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                #Get the distance between the KDP and ZDR centroids
                print(kdp_con_centroid_lon, kdp_con_centroid_lat, zdr_con_centroid_lon, zdr_con_centroid_lat)
                distance_kdp_zdr = g.inv(kdp_con_centroid_lon, kdp_con_centroid_lat, zdr_con_centroid_lon, zdr_con_centroid_lat)
                #print(distance_1[2]/1000)
                print("separation:", distance_kdp_zdr[2])
                dist_kdp_zdr.append(distance_kdp_zdr[2]/1000.)
            except:
                print('storm missing kdp or zdr')
    else:
            print(zdr_storm_lon[i[0]])
            #Find the arc objects associated with this storm:
            zdr_objects_lons = zdr_centroid_lons[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
            zdr_objects_lats = zdr_centroid_lats[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
            print("zdr lons:", zdr_objects_lons)
            print("arc lats:", zdr_objects_lats)
            #Get the sum of their areas
            print(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
            zdr_con_areas.append(np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
            print("Areas sum:",zdr_con_areas)
            #Find the actual centroids
            weighted_lons = zdr_objects_lons * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
            zdr_con_centroid_lon = np.sum(weighted_lons) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
            weighted_lats = zdr_objects_lats * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
            zdr_con_centroid_lat = np.sum(weighted_lats) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
            print("New centroid lon:", zdr_con_centroid_lon, "New centroid lat:", zdr_con_centroid_lat)
            print("lons out of loop", zdr_objects_lons)
            try:
                #Find the kdp objects associated with this storm:
                kdp_objects_lons = kdp_centroid_lons[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                print("kdp lons:", kdp_objects_lons)
                kdp_objects_lats = kdp_centroid_lats[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                #Find the actual centroids
                weighted_lons_kdp = kdp_objects_lons * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                kdp_con_centroid_lon = np.sum(weighted_lons_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                weighted_lats_kdp = kdp_objects_lats * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                print("Could be it:","weighted lons:",weighted_lons_kdp, "object lons",kdp_objects_lons, "areas:",kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                kdp_con_centroid_lat = np.sum(weighted_lats_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                #Get the distance between the KDP and ZDR centroids
                print(kdp_con_centroid_lon, kdp_con_centroid_lat, zdr_con_centroid_lon, zdr_con_centroid_lat)
                distance_kdp_zdr = g.inv(kdp_con_centroid_lon, kdp_con_centroid_lat, zdr_con_centroid_lon, zdr_con_centroid_lat)
                #print(distance_1[2]/1000)
                print("separation:", distance_kdp_zdr[2])
                dist_kdp_zdr.append(distance_kdp_zdr[2]/1000.)
            except:
                print('storm missing kdp or zdr')

In [ ]:
print(dist_kdp_zdr)
print(kdp_objects_lons)
print(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
print(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
weighted_lons_kdp = kdp_objects_lons * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
print(weighted_lons_kdp)
print(np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
print(np.sum(weighted_lons_kdp))
print(np.sum(weighted_lons_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))

In [ ]:
#ZDRmasked3 = ma.masked_where(CCmasked < .85, ZDRmasked2)
fig=plt.figure(2,figsize=(30.,25.))
ax = plt.subplot(111,projection=ccrs.PlateCarree())
ax.coastlines('50m',edgecolor='black',linewidth=0.75)
#ax.add_feature(states_provinces,edgecolor='black',linewidth=0.5)
#Add states/counties and set extent
ax.add_feature(counties, edgecolor = 'black', linewidth = 0.5)
ax.add_feature(states, edgecolor = 'black', linewidth = 1.5)
ax.set_extent([LL[0],UR[0],LL[1],UR[1]])
zdlevs = np.arange(1.75,8,1)
#cf = ax.contourf(rlons[0,:,:],rlats[0,:,:],grad_ffd,grad_levs,cmap = plt.cm.gist_ncar)
#ax.barbs(rlons[0,:,:],rlats[0,:,:],REFgradient[0,:,:],REFgradient[1,:,:],length=10,regrid_shape=12, color = 'k')
ax.contourf(rlons[0,:,:],rlats[0,:,:],ZDRmasked1,zdlevs,cmap = plt.cm.viridis)
ax.contourf(rlons[0,:,:],rlats[0,:,:],ZDRmasked2,zdlevs,cmap = plt.cm.plasma_r, alpha = .7)
for i in enumerate(zdr_areas):
        plt.text(zdr_centroid_lon[i[0]]+.016, zdr_centroid_lat[i[0]]+.016, "%.2f km^2" %(zdr_areas[i[0]].magnitude), size = 23)
if np.asarray(zdr_centroid_lon).shape[0] > 0:
        ax.scatter(zdr_centroid_lon, zdr_centroid_lat, marker = '*', s = 500, color = 'black', zorder = 10, transform=ccrs.PlateCarree())
#ax.scatter(zdr_con_centroid_lon, zdr_con_centroid_lat, marker = '*', s = 500, color = 'orange', zorder = 10, transform=ccrs.PlateCarree())
if len(zdr_con_areas) > 0:
    ax.scatter(zdr_con_centroid_lon, zdr_con_centroid_lat, marker = '*', s = 500, color = 'orange', zorder = 10, transform=ccrs.PlateCarree())
    try:
        plt.text(zdr_con_centroid_lon[i[0]]+.016, zdr_con_centroid_lat[i[0]]+.016, "%.2f km^2" %(zdr_con_areas[i[0]].magnitude), size = 23)
    except:
        print('failed')
        plt.text(float(zdr_con_centroid_lon)+.016, float(zdr_con_centroid_lat)+.016, "%.2f km^2" %(float(zdr_con_areas[0])), size = 23)

#plt.colorbar(cf)
#plt.savefig("DDCbroughttoyoubytechnicolor1.png")
plt.show()

In [ ]:
#ZDRmasked3 = ma.masked_where(CCmasked < .85, ZDRmasked2)
fig=plt.figure(2,figsize=(30.,25.))
ax = plt.subplot(111,projection=ccrs.PlateCarree())
ax.coastlines('50m',edgecolor='black',linewidth=0.75)
#ax.add_feature(states_provinces,edgecolor='black',linewidth=0.5)
#Add states/counties and set extent
ax.add_feature(counties, edgecolor = 'black', linewidth = 0.5)
ax.add_feature(states, edgecolor = 'black', linewidth = 1.5)
ax.set_extent([LL[0],UR[0],LL[1],UR[1]])
zdlevs = np.arange(1.75,8,1)
zdl = [4.0]
kdlevs = np.arange(.75,3, .25)
#cf = ax.contourf(rlons[0,:,:],rlats[0,:,:],grad_ffd,grad_levs,cmap = plt.cm.gist_ncar)
#ax.barbs(rlons[0,:,:],rlats[0,:,:],REFgradient[0,:,:],REFgradient[1,:,:],length=10,regrid_shape=12, color = 'k')
#ax.contourf(rlons[0,:,:],rlats[0,:,:],ZDRmasked,zdlevs,cmap = plt.cm.viridis, zorder = 3, alpha = .76)
#ax.contour(rlons[0,:,:],rlats[0,:,:],ZDRmasked,zdl, colors = 'red', linewidths = 3, zorder = 4)
#ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,kdlevs,cmap = plt.cm.gist_ncar)
ax.scatter(-94.4, 32.79, s=2000, marker = '^', color = 'y', zorder = 5)
#ax.contourf(rlons[0,:,:],rlats[0,:,:],ZDRmasked2,zdlevs,cmap = plt.cm.plasma_r, alpha = .7)
zdrp = ax.pcolormesh(ungrid_lons, ungrid_lats, zdr_c, cmap=plt.cm.nipy_spectral, vmin = -2, vmax = 6)
plt.colorbar(zdrp)
#plt.savefig("DDCbroughttoyoubytechnicolor1.png")
plt.show()

In [ ]:
print(radar.fields)

In [ ]:
proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
               pyproj.Proj(init='epsg:3857'))
zdr_areas = []
zdr_centroid_lon = []
zdr_centroid_lat = []
zdr_mean = []
zdr_max = []

for col in zdrc.collections:
    # Loop through all polygons that have the same intensity level
    for contour_path in col.get_paths(): 
        # Create the polygon for this intensity level
        # The first polygon in the path is the main one, the following ones are "holes"
        for ncp,cp in enumerate(contour_path.to_polygons()):
            cpa = np.asarray(cp[:])
            x = cpa[:,0]
            y = cpa[:,1]
            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
            if ncp == 0:
                poly = new_shape
                print(new_shape)
            else:
                # Remove the holes if there are any
                poly = poly.difference(new_shape)

        # do something with polygon
        #print(poly.area) 
        print(poly.centroid.x)
        s_new = transform(proj, poly)
        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
        boundary = np.asarray(poly.boundary.xy)
        polypath = Path(boundary.transpose())
        coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
        mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
        mean = np.mean(ZDRmasked[mask])
        if projected_area > 20 * units('km^2') and mean > 3.0:
            zdr_areas.append((projected_area))
            zdr_centroid_lon.append((poly.centroid.x))
            zdr_centroid_lat.append((poly.centroid.y))
            zdr_mean.append((mean))
            zdr_max.append((np.max(ZDRmasked[mask])))
        #print(s_new)
print(zdr_areas)
print(zdr_centroid_lon)
print(zdr_centroid_lat)
print(zdr_mean)
print(zdr_max)

In [ ]:
for i in range(len(zdrc.collections)):
    p = zdrc.collections[i].get_paths()[0]
    v = p.vertices
    x = v[:,0]
    y = v[:,1]
    if len(x)>2:
        poly = sp.Polygon([(i[0], i[1]) for i in zip(x,y)])
        print(i, poly.area)

In [ ]:
print(len(zdrc.collections))

In [ ]:
print(enumerate(contour_path.to_polygons()))
for cp in enumerate(contour_path.to_polygons()):
    print(1)

In [ ]:
proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
               pyproj.Proj(init='epsg:3857'))
kdp_areas = []
kdp_centroid_lon = []
kdp_centroid_lat = []

from shapely import geometry
for col in kdpc.collections:
    # Loop through all polygons that have the same intensity level
    for contour_path in col.get_paths(): 
        # Create the polygon for this intensity level
        # The first polygon in the path is the main one, the following ones are "holes"
        for ncp,cp in enumerate(contour_path.to_polygons()):
            print(1)
            cpa = np.asarray(cp[:])
            x = cpa[:,0]
            y = cpa[:,1]
            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
            if ncp == 0:
                poly = new_shape
            else:
                # Remove the holes if there are any
                poly = poly.difference(new_shape)

        # do something with polygon
        #print(poly.area) 
        print(poly.centroid.x)
        s_new = transform(proj, poly)
        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
        if projected_area > 20 * units('km^2'):
            kdp_areas.append((projected_area))
            kdp_centroid_lon.append((poly.centroid.x))
            kdp_centroid_lat.append((poly.centroid.y))

        #print(s_new)
print(kdp_areas)
print(kdp_centroid_lon)
print(kdp_centroid_lat)

In [ ]:
proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
               pyproj.Proj(init='epsg:3857'))
zdr_areas = []
zdr_centroid_lon = []
zdr_centroid_lat = []
zdr_mean = []
zdr_max = []

for col in zdrc.collections:
    # Loop through all polygons that have the same intensity level
    print('hi')
    for contour_path in col.get_paths(): 
        # Create the polygon for this intensity level
        # The first polygon in the path is the main one, the following ones are "holes"
        print('hi')
        for ncp,cp in enumerate(contour_path.to_polygons()):
            print('hi')
            cpa = np.asarray(cp[:])
            x = cpa[:,0]
            y = cpa[:,1]
            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
            if ncp == 0:
                poly = new_shape
                print('hi')
            else:
                # Remove the holes if there are any
                poly = poly.difference(new_shape)
                print('hi')

        # do something with polygon
        #print(poly.area) 
        #print(poly.centroid.x)
        s_new = transform(proj, poly)
        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
        boundary = np.asarray(poly.boundary.xy)
        polypath = Path(boundary.transpose())
        coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
        mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
        mean = np.mean(ZDRmasked[mask])
        if projected_area > 20 * units('km^2') and mean > 3.0:
            zdr_areas.append((projected_area))
            zdr_centroid_lon.append((poly.centroid.x))
            zdr_centroid_lat.append((poly.centroid.y))
            zdr_mean.append((mean))
            zdr_max.append((np.max(ZDRmasked[mask])))
        #print(s_new)
print(zdr_areas)
print(zdr_centroid_lon)
print(zdr_centroid_lat)
print(zdr_mean)
print(zdr_max)

In [ ]:
query = rs.query()
#Here, set the initial time of the archived radar loop you want.
dt = datetime(2017, 7, 11, 23, 30) # Our specified time
station = 'KMVX'
query.stations(station).time_range(dt, dt + timedelta(hours=.1))
cat = rs.get_catalog(query)
cat.datasets
f = 27
n = 1
for item in sorted(cat.datasets.items()):
    # After looping over the list of sorted datasets, pull the actual Dataset object out
    # of our list of items and access over CDMRemote
    try:
        ds = item[1]
        radar = pyart.io.nexrad_cdm.read_nexrad_cdm(ds.access_urls['OPENDAP'])
    except:
        continue

In [ ]:
radar1 = radar.extract_sweeps([1])
print(np.mean(radar1.elevation['data']))

In [ ]:
print(radar.nsweeps)
print(np.max(np.asarray(radar1.fields['differential_reflectivity']['data'])))

In [ ]:
for i in range(radar.nsweeps):
        radar2 = radar.extract_sweeps([i])
        print(np.mean(radar2.elevation['data']))
        if ((np.mean(radar2.elevation['data']) < .52) and (np.max(np.asarray(radar2.fields['differential_reflectivity']['data'])) != 1.0)):
                print(np.mean(radar2.elevation['data']))
                time_start = netCDF4.num2date(radar2.time['data'][0], radar2.time['units'])
                print(time_start)
                kdp_dict = pyart.retrieve.kdp_proc.kdp_maesaka(radar2)
                print('kdp worked')
        